# **LP2_Run1**

In [2]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached glpk-0.4.7.tar.gz (161 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for glpk (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for glpk
Failed to build glpk
ERROR: Could not build wheels for glpk, which is required to install pyproject.toml-based projects


In [3]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [5]:
problem = """You are in charge of the supply purchasing of a company that produces two kinds of drugs.
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market.

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales.

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded.
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available.
Your company can also only store a limited amount of raw materials.
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces.
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively.
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met.  """

In [6]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [7]:
print(response1.choices[0].message.content)

Parameters:

- r1: The cost per unit of material 1 purchased.

- r2: The cost per unit of material 2 purchased.

- d1: The revenue per unit drug 1 produced.

- d2: The revenue per unit drug 2 produced.

- c: The operational cost for drug production.

- b: The budget for purchasing raw materials and operating the production process.

- m: The amount of manpower available.

- e: The number of equipment hours available.

- s: The storage limit for raw materials.

- a1: The amount of active ingredient that can be extracted from a unit of raw material 1.

- a2: The amount of active ingredient that can be extracted from a unit of raw material 2.

- A1: The required amount of active ingredient in drug 1.

- A2: The required amount of active ingredient in drug 2.


- M1: The maximum variation in active ingredient extraction from material 1 (0.5%).

- M2: The maximum variation in active ingredient extraction from material 2 (2%).

Decision Variables:

- x1: The amount of material 1 to be purcha

### **Generate Pyomo Code**

In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [9]:
Markdown(response2.choices[0].message.content)

```python
from pyomo.environ import *

# Step 1: Create a concrete model
model = ConcreteModel()

# Step 2: Define index sets
## All data here are examples
r1 = 3.00
r2 = 4.00
d1 = 20.00
d2 = 15.00
c = 1000
b = 10000
m = 100
e = 80
s = 500
a1 = 0.7
a2 = 0.8
A1 = 0.05
A2 = 0.04
M1 = 0.005
M2 = 0.02
## You can change example data in the future

# Step 3: Define the variables
model.x1 = Var(domain=NonNegativeReals) #Amount of material 1 to be purchased
model.x2 = Var(domain=NonNegativeReals) #Amount of material 2 to be purchased
model.y1 = Var(domain=NonNegativeReals) #units of drug 1 to be produced
model.y2 = Var(domain=NonNegativeReals) #units of drug 2 to be produced
      
# Step 4: Define the objective
model.profit = Objective(
    expr = (d1*model.y1 + d2*model.y2) - (r1*model.x1 + r2*model.x2 + c),
    sense = maximize)

# Step 5: Define the Constraints
model.budget_constraint = Constraint(
    expr = r1*model.x1 + r2*model.x2 + c <= b)

model.manpower_equipment_constraint = Constraint(
    expr = model.y1 + model.y2 <= min(m, e))

model.storage_constraint = Constraint(
    expr = model.x1 + model.x2 <= s)

model.active_ingredient_constraint = Constraint(
    expr = a1*model.x1*(1-M1) + a2*model.x2*(1-M2) >= A1*model.y1 + A2*model.y2)

# Step 6: Solve the model
solver = SolverFactory('glpk')
solver.solve(model)


# print the results
print("The amount of material 1 to be purchased: ", model.x1())
print("The amount of material 2 to be purchased: ", model.x2())
print("The units of drug 1 to be produced: ", model.y1())
print("The units of drug 2 to be produced: ", model.y2())
print("Objective (Max Profit): ", model.profit())
```
This example code use GLPK as a solver. This solver is for linear programing problems. Note that you need to install this solver in your environment before using it. You can replace it with other solvers such as cplex or gurobi if your optimization model is not a linear programming. Pyomo supports many solvers so that you can choose the most suitable for each optimization problem.

### **Run the code Generated by GPT4**

In [12]:
from pyomo.environ import *

# Step 1: Create a concrete model
model = ConcreteModel()

# Step 2: Define index sets
## All data has been inputted by human
r1 = 100
r2 = 199.90
d1 = 6200
d2 = 6900
c = 750 # took average of 700 and 800, are no differentation was made by the math model between cost of producing drug 1 and 2.
b = 100000
m = 2000
e = 800
s = 1000
a1 = 0.01
a2 = 0.02
A1 = 0.5
A2 = 0.6
M1 = 0.005
M2 = 0.02
## END

# Step 3: Define the variables
model.x1 = Var(domain=NonNegativeReals) #Amount of material 1 to be purchased
model.x2 = Var(domain=NonNegativeReals) #Amount of material 2 to be purchased
model.y1 = Var(domain=NonNegativeReals) #units of drug 1 to be produced
model.y2 = Var(domain=NonNegativeReals) #units of drug 2 to be produced

# Step 4: Define the objective
model.profit = Objective(
    expr = (d1*model.y1 + d2*model.y2) - (r1*model.x1 + r2*model.x2 + c),
    sense = maximize)

# Step 5: Define the Constraints
model.budget_constraint = Constraint(
    expr = r1*model.x1 + r2*model.x2 + c <= b)

model.manpower_equipment_constraint = Constraint(
    expr = model.y1 + model.y2 <= min(m, e))

model.storage_constraint = Constraint(
    expr = model.x1 + model.x2 <= s)

model.active_ingredient_constraint = Constraint(
    expr = a1*model.x1*(1-M1) + a2*model.x2*(1-M2) >= A1*model.y1 + A2*model.y2)

# Step 6: Solve the model
solver = SolverFactory('glpk')
solver.solve(model)


# print the results
print("The amount of material 1 to be purchased: ", model.x1())
print("The amount of material 2 to be purchased: ", model.x2())
print("The units of drug 1 to be produced: ", model.y1())
print("The units of drug 2 to be produced: ", model.y2())
print("Objective (Max Profit): ", model.profit())

The amount of material 1 to be purchased:  992.5
The amount of material 2 to be purchased:  0.0
The units of drug 1 to be produced:  19.75075
The units of drug 2 to be produced:  0.0
Objective (Max Profit):  22454.649999999994


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**